In [18]:
import numpy as np


In [38]:
# network size N
N = 5

# number of edges
M = N*3

## initialize time
T = 0
## choose Tmax
Tmax = 5

# network size N
N = 5

# number of edges
M = N*3

Tmax = 5


In [47]:
class Counter(dict):
    def increment(self, item, delta = 1):
        new_val = delta + self.pop(item, 0)
        if new_val > 0:
            self[item] = new_val

def find_tranformation_matrix(b, E):

    ## find the number of events that occur in the interval from T = 0 to T = Tmax
    n = np.random.poisson(lam = Tmax*(N+b*M))

    ## find times for all events
    times = np.sort(np.random.uniform(0, Tmax, n))

    ## initialize boolean identity Matrix A of dimensions NxN
    X = np.identity(N, dtype=bool)
    
    vf = np.vectorize(lambda x: 2**x)
    ints = np.array(list(range(N)))
    I = vf(ints)[::-1]

    counts = Counter()
    for element in I:
        counts.increment(element, 1)


    distinct_rows = N

    # run simulation
    for t in range(n):
        
        if np.random.random() < N/(N + b*M):

            i = np.random.randint(N)

            print(f"recovery event at iteration {t} of node {i}")
        
            row_i_not_distinct = I[i] in counts and counts[I[i]] > 1


            if I[i] != 0:

                if 0 in counts and counts[0] >= 1:

                    if row_i_not_distinct == False:

                        distinct_rows -= 1

                     
                else:

                    if row_i_not_distinct == True:
                            
                        distinct_rows = min(distinct_rows + 1, N)
                
                counts.increment(I[i], -1)

                X[i, :] = False

                I[i] = 0

                counts.increment(0, 1)
        else:

            i,j = E[np.random.randint(M)]

            print(f"spreading event at iteration {t} of nodes {i} and {j}")

            if (I[i] != 0 or I[j] != 0) and I[i] != I[j]:
                    
                new = int(''.join('1' if x else '0' for x in np.logical_or(X[i], X[j])), 2)
                counts.increment(I[i], -1)
                counts.increment(I[j], -1)
                
                new_not_distinct = new in counts and counts[new] >= 1
                print(new_not_distinct)
                    
                row_i_not_distinct = I[i] in counts and counts[I[i]] >= 1
                print(row_i_not_distinct)

                row_j_not_distinct = I[j] in counts and counts[I[j]] >= 1
                print(row_j_not_distinct)

                counts.increment(new, 2)
                I[i] = I[j] = new

                if new_not_distinct == False:
                    if row_i_not_distinct == False and row_j_not_distinct == False:
                        distinct_rows -= 1
                    if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
                        distinct_rows = distinct_rows
                    if row_i_not_distinct == True and row_j_not_distinct == True:
                        distinct_rows = min(distinct_rows + 1, N)

                else:
                    if row_i_not_distinct == False and row_j_not_distinct == False:
                        distinct_rows -= 2
                    if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
                        distinct_rows -= 1

                X[i] = X[j] = np.logical_or(X[i], X[j])

            else:
                print("no spread possible")

        print(distinct_rows)
        ## break the simulation because the pseudo mixing time has been found (the number of distinct rows and thus distinct columns is 2)
        if distinct_rows == 2:
            
            break

    return X, I


just do len(counts) rather than add/ subtract from disctinct_rows()
len is very quick! 
using python dictionary that keeps track of metadata about itself automatically. 

In [ ]:
class Counter(dict):
    def increment(self, item, delta = 1):
        new_val = delta + self.pop(item, 0)
        if new_val > 0:
            self[item] = new_val

def find_tranformation_matrix(b, E):

    ## find the number of events that occur in the interval from T = 0 to T = Tmax
    n = np.random.poisson(lam = Tmax*(N+b*M))

    ## find times for all events
    times = np.sort(np.random.uniform(0, Tmax, n))

    ## initialize boolean identity Matrix A of dimensions NxN
    X = np.identity(N, dtype=bool)
    
    vf = np.vectorize(lambda x: 2**x)
    ints = np.array(list(range(N)))
    I = vf(ints)[::-1]

    counts = Counter()
    for element in I:
        counts.increment(element, 1)


    distinct_rows = N

    # run simulation
    for t in range(n):
        
        if np.random.random() < N/(N + b*M):

            i = np.random.randint(N)

            print(f"recovery event at iteration {t} of node {i}")


            if I[i] != 0:
                
                counts.increment(I[i], -1)

                I[i] = 0

                counts.increment(0, 1)
        else:

            i,j = E[np.random.randint(M)]

            print(f"spreading event at iteration {t} of nodes {i} and {j}")

            if (I[i] != 0 or I[j] != 0) and I[i] != I[j]:
                    
                new = I[i] | I[j]
                counts.increment(I[i], -1)
                counts.increment(I[j], -1)

                counts.increment(new, 2)
                I[i] = I[j] = new

        distinct_rows = len(counts)
        ## break the simulation because the pseudo mixing time has been found (the number of distinct rows and thus distinct columns is 2)
        if distinct_rows == 2:
            
            break

    return I


In [48]:
b = 0.5

## random edge matrix
E = np.random.randint(0,N,size=2*M).reshape((N*3, 2))

x0 = np.ones(N, dtype=bool)
matrix_A = find_tranformation_matrix(b, E)
print(matrix_A)


recovery event at iteration 0 of node 3
4
spreading event at iteration 1 of nodes 3 and 3
4
spreading event at iteration 2 of nodes 2 and 3
True
False
False
2
(array([[ True, False, False, False, False],
       [False,  True, False, False, False],
       [False, False,  True, False, False],
       [False, False,  True, False, False],
       [False, False, False, False,  True]]), array([ 1,  2,  4,  4, 16]))


In [21]:
# Your decimal number
decimal_number = 5

# The length of the binary string
length = 8

# Convert the decimal number to a binary string of a specific length
binary_string = format(decimal_number, '0' + str(length) + 'b')
binary_array = np.array([int(bit) for bit in binary_string])
print(binary_array)



[0 0 0 0 0 1 0 1]


In [26]:
vf = np.vectorize(lambda x: 2**x)
X = np.array(list(range(N)))
I = vf(X)
print(I)


[ 1  2  4  8 16]


In [37]:
class Counter(dict):
    def increment(self, item, delta = 1):
        new_val = delta + self.pop(item, 0)
        if new_val > 0:
            self[item] = new_val

counts = Counter()
for element in I:
    counts.increment(element, 1)

counts.increment(1, 1)
print(len(counts))
print(counts)
print(counts[1])


I[1]
row_i_not_distinct = I[1] in counts and counts[I[1]] > 1
print(row_i_not_distinct)

counts[0] >= 1


5
{2: 1, 4: 1, 8: 1, 16: 1, 1: 2}
2
False


KeyError: 0